# Get Book IDs for Fantasy Books

In [1]:
#!pip install goodreads-api-client
#!pip install xmltodict
import goodreads_api_client as gr
from goodreads import client
import requests

In [2]:
from bs4 import BeautifulSoup
import xmltodict
import pprint
import json
import os
#!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

**Goodreads API credentials**

In [3]:
api_key = 'Jjd519OAM82QfNQGepZsw'
api_secret = 'JY9m0mBvG7U1Ws4AUkpMucbwCNi6tndGTIqdASYo'

In [8]:
client = gr.Client(developer_key=api_key)

from goodreads import client
gc = client.GoodreadsClient(api_key, api_secret)

**Define helpful functions**

In [4]:
def get_info(url):
    key = api_key
    response = requests.get(url, params={'key': api_key})
    #soup = BeautifulSoup(response.content, "lxml-xml")
    results = xmltodict.parse(response.content)
    #results = json.dumps(results)
    return(results['GoodreadsResponse'])

In [11]:
def get_soup(url):
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    #print(soup.prettify())
    return(soup)

def get_soup_selenium(url):
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    return(soup)
def stripHTML(v):
    v = [re.sub(r'\t', '', x) for x in v]
    v = [re.sub(r'\r', '', x) for x in v]
    v = [re.sub(r'\n', '', x) for x in v]
    return(v)

**Links to all listopia links**

In [12]:
main_url = 'https://www.goodreads.com/list/tag/fantasy'
soup = get_soup(main_url)
links = soup.find_all('a', href=True)
len(links)

339

In [13]:
list_links = []
for line in soup.find_all('a'):
    try:
        link = line['href']
        if 'list/show' in link:
            list_links.append('https://www.goodreads.com'+str(link))
    except:
        pass
list_links = list(set(list_links))
len(list_links)

30

In [16]:
list_links

['https://www.goodreads.com/list/show/211.Best_Time_Travel_Fiction',
 'https://www.goodreads.com/list/show/26495.Best_Woman_Authored_Books',
 'https://www.goodreads.com/list/show/642.Hidden_Gems_YA_Fantasy_Novels',
 'https://www.goodreads.com/list/show/3.Best_Science_Fiction_Fantasy_Books',
 'https://www.goodreads.com/list/show/51.The_Best_Urban_Fantasy',
 'https://www.goodreads.com/list/show/559.What_To_Read_After_Harry_Potter',
 'https://www.goodreads.com/list/show/25529.Best_Unknown_but_must_be_Known_books_',
 'https://www.goodreads.com/list/show/1023.Best_Strong_Female_Fantasy_Novels',
 'https://www.goodreads.com/list/show/4252.What_Book_Would_You_Like_To_Live_In_',
 'https://www.goodreads.com/list/show/7057.Original_Stories_a_Breath_of_Fresh_Air',
 'https://www.goodreads.com/list/show/618.Best_Steampunk_Books',
 'https://www.goodreads.com/list/show/871.Most_Interesting_Magic_System',
 'https://www.goodreads.com/list/show/319.Best_Fantasy_Series_Trilogies_and_Duologies',
 'https://

**Functions to scrape ids for books on fantasy listopia**

In [18]:
def get_page_max(soup):
    page_links=[]
    for line in soup.find_all('a'):
        try:
            # Go to the next page while under max_pages
            page_url = url.split('goodreads.com')[1]+'?page='
            if page_url in line['href']:
                page_links.append(line['href'])    
        except:   
            pass
    max_pages = int(page_links[-2].split('=')[1])
    return(max_pages)

def get_book_ids(soup):
    ids = []
    tables = soup.find_all("table")
    rows = tables[0].find_all('tr')
    tds = [row.find_all('td') for row in rows]

    for i in range(len(tds)):
        try:
            ids.append(tds[i][1].find("div", {"class" :"u-anchorTarget"})['id'])
        except:
            pass
    return(ids)

**Loop through each link and save book ids to list**

In [19]:
soup_success = 0
soup_fail = 0

ids = []

# Loop through all fantasy book links
for i in range(len(list_links)):
    # Open each link with fantasy book lists
    url = list_links[i]
    try:
        soup = get_soup(url)
        # Get book id's from page 1
        ids = ids+get_book_ids(soup)
        # Go through remaining pages up to page max
        pages = get_page_max(soup)
        for p in range(2, pages):
            try:
                soupp = get_soup(url+'?page='+str(p))
                ids = ids+get_book_ids(soupp)
            except:
                pass        
        soup_success +=1
    except:
        soup_fail+=1
len(ids)

83000

**Save book ids to pickle**

In [20]:
import pickle

with open('book_ids.pkl', 'wb') as f:
    pickle.dump(ids, f)

**Set of unique bookids**

In [25]:
len(list(set(ids)))

46057

**END NOTEBOOK**

***